In [2]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import re
import os
from openpyxl import load_workbook
from shutil import move
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait

In [3]:
url="https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page={}&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_{}"
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument('headless')
# options.add_argument('Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like Mac OS X) AppleWebKit/604.3.5 (KHTML, like Gecko) Version/13.0 MQQBrowser/10.1.1 Mobile/15B87 Safari/604.1 QBWebViewUA/2 QBWebViewType/1 WKType/1')
chrome = webdriver.Chrome('./chromedriver')
data = []
tStart = time.time()
for i in range(1,3):
    chrome.get(url.format(i,i))
    chrome.find_element_by_id('twotabsearchtextbox').send_keys(Keys.F12)
    print(url.format(i,i))
#     time.sleep(5)
    soup = BeautifulSoup(chrome.page_source,'lxml')
    amazon_card=soup.find_all(class_="a-size-medium a-color-base a-text-normal")
    if len(amazon_card) == 0:
        print("找不到顯示卡標題")
        break
    data.append([ele.text for ele in amazon_card])   
chrome.close()    
tEnd = time.time()
print("總共耗時:%f 秒" % (tEnd-tStart))

https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page=1&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_1
https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page=2&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_2
總共耗時:11.203626 秒


In [4]:
amazon_json = json.load(open("AMAZON_COMPUTER_PRICE" + ".json",encoding="utf-8"))
print(amazon_json['title'])

['Asus', 'MSI', 'Sapphire', 'PowerColor', 'MSI微星', 'NVIDIA', 'ASUS', 'Gigabyte', 'ZOTAC', 'XFX', 'EVGA']


In [5]:
title=[]
for adx,datas in enumerate(data):
    
    df = 'df'+str(adx) 
    df = pd.DataFrame(datas,columns=['f'])
    columns_of_datas = [ddd for ddd in range(0,len(datas))]  
    # 分種類並寫入json中
    for aaa in columns_of_datas:
    # 正則表達式將標題的牌子切割出來
        matchObj = re.split(r'\s', df['f'][aaa], re.M|re.I)
        title.append(matchObj[0])
# 篩選重複的牌子        
json_title_dumps = list(set(title))
json_title = {'title': json_title_dumps}
amazon_json.update(json_title)
# 將整理好title的json寫入json檔案中
with open('AMAZON_COMPUTER_PRICE.json', 'w', encoding='utf-8') as f:
    json.dump(amazon_json, f, ensure_ascii=False, indent=4,sort_keys=True)
print(amazon_json)

{'sheet_name': '顯示卡', 'title': ['EVGA', 'ZOTAC', 'ASUS', 'NVIDIA', 'MSI微星', 'XFX', 'Gigabyte', 'MSI', 'PowerColor', 'Sapphire']}


In [13]:
now_date = datetime.strftime(datetime.now() , '%Y%m%d')
amazon_json = json.load(open("AMAZON_COMPUTER_PRICE" + ".json",encoding="utf-8"))
# 沒檔案的話則建立
output_file = now_date + "_amazon_graphics card.xlsx"
target_path = "D:\\Amazon_graphics_card\\"
if os.path.exists(output_file) == False:
    writer = pd.ExcelWriter(output_file, engine='openpyxl')
    writer.book.create_sheet('default')  # 新增空sheet
else:  # 如果檔案存在，append
    book = load_workbook(output_file)
    writer = pd.ExcelWriter(output_file, engine='openpyxl')
    writer.book = book
    
if amazon_json['sheet_name'] in writer.book.sheetnames:
    writer.book.remove(writer.book[amazon_json['sheet_name']])
    

for adx,datas in enumerate(data):
    df = "df" + str(adx)    
    df = pd.DataFrame(datas) 
    df.to_excel(writer, index=False) 


# 寫入excel sheet
  
if (len(writer.book.sheetnames) > 1):  # 若有其他sheet，刪除default sheet
    if 'default' in writer.book.sheetnames:
        writer.book.remove(writer.book['default'])    
writer.save()  


In [229]:
cwd = os.getcwd()
curr_file = cwd + "\\" + output_file
target_file = target_path + output_file

try:
    move(curr_file, target_file)
    print(output_file + "\n移動檔案完畢.")
except Exception as e:
    print(output_file + "\n移動檔案失敗.")
    print(e.args)

20200916_amazon_graphics card.xlsx
移動檔案完畢.


In [55]:
# 看這裡
def title_var(title,datas,data_len):
    for i in data_len:
       a = [datas[i] for s_title in title if s_title in datas[i]]
#         if title in datas[i]:
#             pop={
#                 amazon_json['title']:datas[i]
#             }
for adx,datas in enumerate(data):
    df = 'df'+str(adx) 
#     每頁資料的數量
    columns_of_datas = [ddd for ddd in range(0,len(datas))]   
    title_var(amazon_json['title'],datas,columns_of_datas)
    
     
#   將每頁資料歸納種類  
    df = pd.DataFrame(columns=amazon_json['title'],data=np.array( [['1','1','1','1','1','1','1','1','1','1']] ))


['EVGA', 'ZOTAC', 'ASUS', 'NVIDIA', 'MSI微星', 'XFX', 'Gigabyte', 'MSI', 'PowerColor', 'Sapphire']
XFX HDMI 顯卡
['EVGA', 'ZOTAC', 'ASUS', 'NVIDIA', 'MSI微星', 'XFX', 'Gigabyte', 'MSI', 'PowerColor', 'Sapphire']
EVGA 08G-P4-3071-KR GeForce RTX 2070 Super Gaming, 8GB GDDR6 處理器


In [164]:
for adx,datas in enumerate(data):   
    r1 = next((datas[0] for x in amazon_json['title'] if x in datas[0]),None)     
    print('r',adx,'=',r1)


r 0 = XFX HDMI 顯卡
r 1 = XFX RX 5700 XT 三重耗 8GB GDDR6 3xDP HDMI 顯示卡 RX-57XT83LD8


In [120]:
df = pd.DataFrame([['A', 2], ['A', 4], ['B', 6]])
print(df.dtypes)
a =df.astype(str)
print('a=',a.dtypes)

0    object
1     int64
dtype: object
a= 0    object
1    object
dtype: object
